<a href="https://colab.research.google.com/github/sgpt94/Games-Recommender/blob/main/ALS_algo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Importing all required modules
from datetime import datetime, timedelta
from statistics import mean

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Gameplay_Data.csv to Gameplay_Data.csv


In [ ]:
import io
game = pd.read_csv(io.BytesIO(uploaded['Gameplay_Data.csv']))

In [ ]:
game.head()

,Uid,Eventtime,Entry_Fee,Win_Loss,Winnings,Tournament_Type,Num_Players
0,90976392B3AA5DB94D134936B765879E,1607294430194,0.0,W,10.0,D,4
1,7457E894E1F7F1938E55802B964BDD27,1607286707669,1000.0,W,1700.0,A,2
2,52E43F75D7393581C4E2C5EDC7D24671,1607023262845,3500.0,L,0.0,A,2
3,5736C06C66C6BC7036CCE9804F54A57C,1607778875873,1000.0,W,3200.0,B,4
4,5FE2DDA792CA72F731D818A5ECC5F1EB,1607778574193,2500.0,W,4200.0,A,2


In [ ]:
game.shape

(307666, 7)

In [ ]:
uploaded = files.upload()

Saving Demographic.csv to Demographic.csv


In [ ]:
demog = pd.read_csv(io.BytesIO(uploaded['Demographic.csv']))
# wallet = pd.read_csv(io.BytesIO(uploaded['Wallet_Balance.csv']))

In [ ]:
demog.shape

(21499, 14)

In [ ]:
demog.head()

,Uid,Installed_At,Connection_Type,Cpu_Type,Network_Type,Device_Manufacturer,ISP,Country,Country_Subdivision,City,Postal_Code,Language,Device_Name,Device_Type
0,24CC6E6BA192D1A1EEB320E77F0BF2EE,1608868973,Cellular,arm64-v8a,0,Realme,Airtel,in,Karnataka,Bengaluru,560002,en,XT,phone
1,2762F8FB0C406E59DC9D665A4CCEE468,1608869117,Cellular,armeabi-v7a,0,Tecno,Jio,in,Bihar,Patna,800001,en,TECNOKD7h,phone
2,0CF35DBD42AC7013E1A295B5C8E78B0A,1608869225,Cellular,arm64-v8a,0,Xiaomi,Idea Cellular,in,Tamil Nadu,Chennai,600012,en,MiA3,phone
3,E829EB3C5A9890718794301564007D91,1608869281,Cellular,arm64-v8a,13,vivo,Vodafone India,in,Tamil Nadu,Salem,636007,en,vivo1820,phone
4,5BF3652D06CFAF7E811416F1647E597F,1608869379,Cellular,arm64-v8a,13,Oppo,Jio,in,National Capital Territory of Delhi,New Delhi,110008,en,A5s,phone


In [ ]:
game.head()

,Uid,Eventtime,Entry_Fee,Win_Loss,Winnings,Tournament_Type,Num_Players
0,90976392B3AA5DB94D134936B765879E,1607294430194,0.0,W,10.0,D,4
1,7457E894E1F7F1938E55802B964BDD27,1607286707669,1000.0,W,1700.0,A,2
2,52E43F75D7393581C4E2C5EDC7D24671,1607023262845,3500.0,L,0.0,A,2
3,5736C06C66C6BC7036CCE9804F54A57C,1607778875873,1000.0,W,3200.0,B,4
4,5FE2DDA792CA72F731D818A5ECC5F1EB,1607778574193,2500.0,W,4200.0,A,2


In [ ]:
# no. of users in demog database
len(demog['Uid'].unique())

21437

In [ ]:
#unique users who have a game
len(game['Uid'].unique())

10499

### 10499 users out of 21437 users have played atleast a single game.<br>
We will consider the users who have played atleast 1 game

In [ ]:
#pivot of Tournament_types vs Entry fees
pd.crosstab(game['Tournament_Type'],game['Entry_Fee'])

Entry_Fee,0.0,100.0,500.0,1000.0,1500.0,2500.0,3500.0,5000.0,10000.0,25000.0,50000.0,100000.0,500000.0,1000000.0
Tournament_Type,,,,,,,,,,,,,,
A,265,113056,0,34528,0,14556,1876,6863,6149,2084,756,481,24,1
B,0,0,0,6862,0,0,0,0,0,0,0,0,0,0
C,0,0,0,16753,0,866,4559,2058,3038,0,0,0,0,0
D,63325,0,26658,0,2908,0,0,0,0,0,0,0,0,0


We can clearly see that Tournament A is the most popular one, with Entry_fee of 100 to be most popular choice

In [ ]:
#removing the games with entry fee above 10K from our model, as those games are very less in 
#number and we would not want to recommend games with very high entry fees to the users
game = game[game['Entry_Fee']<25000]

In [ ]:
game.shape

(304320, 8)

In [ ]:
#creating a column combining Tournament_type and Entry_fee
game['type'] = game.apply(lambda x: str(x['Tournament_Type'])+'_'+str(int(x['Entry_Fee'])),axis=1)

In [ ]:
game.head()

,Uid,Eventtime,Entry_Fee,Win_Loss,Winnings,Tournament_Type,Num_Players,type
0,90976392B3AA5DB94D134936B765879E,1607294430194,0.0,W,10.0,D,4,D_0
1,7457E894E1F7F1938E55802B964BDD27,1607286707669,1000.0,W,1700.0,A,2,A_1000
2,52E43F75D7393581C4E2C5EDC7D24671,1607023262845,3500.0,L,0.0,A,2,A_3500
3,5736C06C66C6BC7036CCE9804F54A57C,1607778875873,1000.0,W,3200.0,B,4,B_1000
4,5FE2DDA792CA72F731D818A5ECC5F1EB,1607778574193,2500.0,W,4200.0,A,2,A_2500


In [ ]:
game['type'].unique()

array(['D_0', 'A_1000', 'A_3500', 'B_1000', 'A_2500', 'A_100', 'C_1000',
       'D_500', 'A_10000', 'C_10000', 'C_3500', 'A_5000', 'C_2500',
       'D_1500', 'C_5000', 'A_0'], dtype=object)

In [ ]:
# total 16 unique types of game types
len(game['type'].unique())

16

Creating a dataframe containing no. of games played by each user in each game_type

In [ ]:
data =  game.groupby(['Uid','type'],as_index=False)['Win_Loss'].size()

In [ ]:
data.rename(columns = {'Uid':'user','size':'plays','type':'game'}, inplace = True)

In [ ]:
data.head()

,user,game,plays
0,0012CB6DA8CB360BA4ACC6A40F2C5E7D,A_100,30
1,0012CB6DA8CB360BA4ACC6A40F2C5E7D,A_1000,3
2,0012CB6DA8CB360BA4ACC6A40F2C5E7D,D_0,16
3,0016F36E9D87DC1C1D8FCA7F84648122,A_100,58
4,0016F36E9D87DC1C1D8FCA7F84648122,A_1000,5


In [ ]:
data.shape

(31265, 3)

In [ ]:
data['plays'].sum()

304320

In [ ]:
#Avg. no. of games played by users
data['plays'].sum()/len(data['user'].unique())

28.98561767787408

In [ ]:
#Avg. no. of different game_types tried by users
data.shape[0]/len(data['user'].unique())

2.9779026573959424

So each user has played on average 29 games and tried 3 different game_types

### Creating a recommendation system using Implicit Alternating Least Square (ALS) Matrix-factorization method

For that we need to first create a sparse matrix of user-games with cell-values as total no. of games played by users in each game-type

In [ ]:
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Convert game names into numerical IDs
data['user_id'] = data['user'].astype("category").cat.codes
data['game_id'] = data['game'].astype("category").cat.codes
 
 # Create a lookup frame so we can get the game names back in 
 # readable form later.
item_lookup = data[['game_id', 'game']].drop_duplicates()
item_lookup['game_id'] = item_lookup.game_id.astype(str)
 
data = data.drop(['user', 'game'], axis=1)
 
 # Drop any rows that have 0 plays
data = data.loc[data.plays != 0]
 
 # Create lists of all Uids, games and plays
users = list(np.sort(data.user_id.unique()))
games = list(np.sort(data.game_id.unique()))
plays = list(data.plays)
 
 # Get the rows and columns for our new matrix
rows = data.user_id.astype(int)
cols = data.game_id.astype(int)
 
 # Contruct a sparse matrix for our Uids and items containing number of plays
data_sparse = sparse.csr_matrix((plays, (rows, cols)), shape=(len(users), len(games)))

NameError: ignored

The sparse matrix is created, now the implicit_ALS algo can return the factorized matrices - user_vecs & item_vecs which is obtained from the sparse matrix as shown below - 

In [ ]:
def nonzeros(m, row):
    for index in range(m.indptr[row], m.indptr[row+1]):
        yield m.indices[index], m.data[index]
      
 #defining the als method     
def implicit_als_cg(Cui, features=20, iterations=20, lambda_val=0.1):
    user_size, item_size = Cui.shape

    X = np.random.rand(user_size, features) * 0.01
    Y = np.random.rand(item_size, features) * 0.01

    Cui, Ciu = Cui.tocsr(), Cui.T.tocsr()

    for iteration in range(iterations):
        print ('iteration %d of %d' % (iteration+1, iterations))
        least_squares_cg(Cui, X, Y, lambda_val)
        least_squares_cg(Ciu, Y, X, lambda_val)
    
    return sparse.csr_matrix(X), sparse.csr_matrix(Y)
  
  
def least_squares_cg(Cui, X, Y, lambda_val, cg_steps=3):
    users, features = X.shape
    
    YtY = Y.T.dot(Y) + lambda_val * np.eye(features)

    for u in range(users):

        x = X[u]
        r = -YtY.dot(x)

        for i, confidence in nonzeros(Cui, u):
            r += (confidence - (confidence - 1) * Y[i].dot(x)) * Y[i]

        p = r.copy()
        rsold = r.dot(r)

        for it in range(cg_steps):
            Ap = YtY.dot(p)
            for i, confidence in nonzeros(Cui, u):
                Ap += (confidence - 1) * Y[i].dot(p) * Y[i]

            alpha = rsold / p.dot(Ap)
            x += alpha * p
            r -= alpha * Ap

            rsnew = r.dot(r)
            p = r + (rsnew / rsold) * p
            rsold = rsnew

        X[u] = x

alpha_val = 15
conf_data = (data_sparse * alpha_val).astype('double')

#creatingthe user and item vecs from the original sparse matrix
user_vecs, item_vecs = implicit_als_cg(conf_data, iterations=20, features=20)

In [ ]:
data.sample(5)

Finding similar items (game_types) - 

In [ ]:

# FIND SIMILAR ITEMS


# Let's find similar games to 'D_500' (item_id = 15). 

item_id = 15

# Get the item row for 'D_500'
item_vec = item_vecs[item_id].T

# Calculate the similarity score 
# and select the top 10 most similar.
scores = item_vecs.dot(item_vec).toarray().reshape(1,-1)[0]
top_10 = np.argsort(scores)[::-1][:10]

games = []
game_scores = []

# Get and print the actual games names and scores
for idx in top_10:
    games.append(item_lookup.game.loc[item_lookup.game_id == str(idx)].iloc[0])
    game_scores.append(scores[idx])

similar = pd.DataFrame({'game': games, 'score': game_scores})

print(similar)

Generating user recommendations - 

In [ ]:
# Let's say we want to recommend games for user with ID 2660
user_id = 2228


# GET ITEMS CONSUMED BY USER


# Let's print out what the user has played
consumed_idx = data_sparse[user_id,:].nonzero()[1].astype(str)
consumed_items = item_lookup.loc[item_lookup.game_id.isin(consumed_idx)]
print(consumed_items)


In [ ]:

# CREATE USER RECOMMENDATIONS


def recommend(user_id, data_sparse, user_vecs, item_vecs, item_lookup, num_items=10):
    """Recommend items for a given user given a trained model    
    """
  
    # Get all interactions by the user
    user_interactions = data_sparse[user_id,:].toarray()

    # We don't want to recommend items the user has consumed. So let's
    # set them all to 0 and the unknowns to 1.
    user_interactions = user_interactions.reshape(-1) + 1 #Reshape to turn into 1D array
    user_interactions[user_interactions > 1] = 0

    # This is where we calculate the recommendation by taking the 
    # dot-product of the user vectors with the item vectors.
    rec_vector = user_vecs[user_id,:].dot(item_vecs.T).toarray()

    # Let's scale our scores between 0 and 1 to make it all easier to interpret.
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0]
    recommend_vector = user_interactions*rec_vector_scaled
   
    # Get all the game indices in order of recommendations (descending) and
    # select only the top "num_items" items. 
    item_idx = np.argsort(recommend_vector)[::-1][:num_items]

    games = []
    scores = []

    # Loop through our recommended game indicies and look up the actual game_types
    for idx in item_idx:
        games.append(item_lookup.game.loc[item_lookup.game_id == str(idx)].iloc[0])
        scores.append(recommend_vector[idx])

    # Create a new dataframe with recommended game_types and scores
    recommendations = pd.DataFrame({'game': games, 'score': scores})
    
    return recommendations

# Let's generate and print our recommendations
recommendations = recommend(user_id, data_sparse, user_vecs, item_vecs, item_lookup)
print(recommendations)